In [1]:
import Pkg; Pkg.activate(@__DIR__); 

# Note: uncomment on FIRST RUN ONLY
# Pkg.instantiate(); 
# Pkg.add(url="https://github.com/JuliaRobotics/RigidBodyDynamics.jl")
# Pkg.update();

using RobotDynamics, Rotations
using TrajectoryOptimization
using StaticArrays, LinearAlgebra
using RigidBodyDynamics
import ForwardDiff;
const RBD = RigidBodyDynamics;
const TO = TrajectoryOptimization;

 Activating environment at `~/funnels/Project.toml`


In [2]:
println(@__DIR__)

/home/sam/funnels


In [3]:
using Random
traj_folder = " ~/.julia/packages/RobotZoo/NBLvF/src"
urdf_folder = joinpath(traj_folder, "urdf")
urdf_kuka_orig = joinpath(urdf_folder, "kuka_iiwa.urdf")
urdf_kuka = joinpath(urdf_folder, "temp","kuka.urdf")

function write_kuka_urdf()
    kuka_mesh_dir = joinpath(urdf_folder,"kuka_iiwa_mesh")
    temp_dir = joinpath(urdf_folder, "temp")
    if !isdir(temp_dir)
        mkdir(temp_dir)
    end
    open(urdf_kuka_orig,"r") do f
        open(urdf_kuka, "w") do fnew
            for ln in eachline(f)
                pre = findfirst("<mesh filename=",ln)
                post = findlast("/>",ln)
                if !(pre isa Nothing) && !(post isa Nothing)
                    inds = pre[end]+2:post[1]-2
                    pathstr = ln[inds]
                    file = splitdir(pathstr)[2]
                    ln = ln[1:pre[end]+1] * joinpath(kuka_mesh_dir,file) * ln[post[1]-1:end]
                end
                println(fnew,ln)
            end
        end
    end
end

function get_kuka_ee(kuka)
    ee_body = findbody(kuka, "iiwa_link_ee")
    ee_point = Point3D(default_frame(ee_body),0.,0.,0.)
    return ee_body, ee_point
end

function get_kuka_ee_postition_fun(kuka::Mechanism,statecache=StateCache(kuka)) where {O}
    ee_body, ee_point = get_kuka_ee(kuka)
    world = root_frame(kuka)
    nn = num_positions(kuka)

    function ee_position(x::AbstractVector{T}) where T
        state = statecache[T]
        RBD.set_configuration!(state, x[1:nn])
        RBD.transform(state, ee_point, world).v
    end
end

# function calc_ee_position(kuka::Mechanism,X::Trajectory)
#     ee = zero.(X)
#     N = length(X)
#     state = MechanismState(kuka)
#     world = root_frame(kuka)
#     ee_point = get_kuka_ee(kuka)[2]
#     nn = num_positions(kuka)
#     for k = 1:N
#         set_configuration!(state, X[k][1:nn])
#         ee[k] = RigidBodyDynamics.transform(state, ee_point, world).v
#     end
#     return ee
# end


function kuka_ee_ik(kuka::Mechanism,point::Vector,ik_iterations=1000,attempts=20,tol=1e-2)
    state = MechanismState(kuka)
    world = root_frame(kuka)

    # Get end-effector
    ee_body, ee_point = get_kuka_ee(kuka)

    # Run IK
    err = Inf
    iter = 1
    while err > tol
        rand!(state)
        goal = Point3D(world,point)
        ik_res = jacobian_transpose_ik!(state,ee_body,ee_point,goal,iterations=ik_iterations)
        point_res = RigidBodyDynamics.transform(ik_res,ee_point,world).v
        err = norm(point-point_res)
        if iter > attempts
            error("IK cannot get sufficiently close to the goal")
        end
        return ik_res
    end
end


function jacobian_transpose_ik!(state::MechanismState,
                               body::RobotDynamics.RigidBody,
                               point::Point3D,
                               desired::Point3D;
                               α=0.1,
                               iterations=100)
    mechanism = state.mechanism
    world = root_frame(mechanism)

    # Compute the joint path from world to our target body
    p = path(mechanism, root_body(mechanism), body)
    # Allocate the point jacobian (we'll update this in-place later)
    Jp = point_jacobian(state, p, RigidBodyDynamics.transform(state, point, world))

    q = copy(configuration(state))

    for i in 1:iterations
        # Update the position of the point
        point_in_world = RigidBodyDynamics.transform(state, point, world)
        # Update the point's jacobian
        point_jacobian!(Jp, state, p, point_in_world)
        # Compute an update in joint coordinates using the jacobian transpose
        Δq = α * Array(Jp)' * (RigidBodyDynamics.transform(state, desired, world) - point_in_world).v
        # Apply the update
        q .= configuration(state) .+ Δq
        set_configuration!(state, q)
    end
    state
end

function hold_trajectory(n,m,N, mech::Mechanism, q)
    state = MechanismState(mech)
    nn = num_positions(state)
    set_configuration!(state, q[1:nn])
    vd = zero(state.q)
    u0 = dynamics_bias(state)

    if length(q) > m
        throw(ArgumentError("system must be fully actuated to hold an arbitrary position ($(length(q)) should be > $m)"))
    end
    U0 = zeros(m,N)
    for k = 1:N
        U0[:,k] = u0
    end
    return U0
end

# Write new urdf file with correct absolute paths
# write_kuka_urdf()

# kuka = Model(urdf_kuka)
# end_effector_function = get_kuka_ee_postition_fun(parse_urdf(urdf_kuka,remove_fixed_tree_joints=false))

# kuka_tree::RBD.Mechanism
kuka_tree = parse_urdf("kuka.urdf",remove_fixed_tree_joints=false)
end_effector_function = get_kuka_ee_postition_fun(kuka_tree)
RBD.joints(kuka_tree)

11-element Array{Joint{Float64,JT} where JT<:JointType{Float64},1}:
 Joint "base_to_world": Fixed joint
 Joint "base_joint": Fixed joint
 Joint "iiwa_joint_1": Revolute joint with axis [0.0, 0.0, 1.0]
 Joint "iiwa_joint_2": Revolute joint with axis [0.0, 0.0, 1.0]
 Joint "iiwa_joint_3": Revolute joint with axis [0.0, 0.0, 1.0]
 Joint "iiwa_joint_4": Revolute joint with axis [0.0, 0.0, 1.0]
 Joint "iiwa_joint_5": Revolute joint with axis [0.0, 0.0, 1.0]
 Joint "iiwa_joint_6": Revolute joint with axis [0.0, 0.0, 1.0]
 Joint "iiwa_joint_7": Revolute joint with axis [0.0, 0.0, 1.0]
 Joint "iiwa_joint_ee": Fixed joint
 Joint "tool0_joint": Fixed joint

## Modeling:
We'll model the robot as fully-actuated and unconstrained by its dynamics.

In [4]:
struct Kuka <: TrajectoryOptimization.AbstractModel
    id::Int32
end

RobotDynamics.control_dim(::Kuka) = 7
RobotDynamics.state_dim(::Kuka) = 7

function RobotDynamics.dynamics(model::Kuka, x, u)
    u
end

model = Kuka(0)
size(model)

(7, 7)

## Cost functions
Create cost functions for each cost, plus a container function

First, create a general nonlinear cost function:

In [5]:
abstract type GeneralCostFunction{n,m} <: TO.CostFunction end
is_blockdiag(::GeneralCostFunction) = false
state_dim(::GeneralCostFunction{n}) where n = n
control_dim(::GeneralCostFunction{<:Any,m}) where m = m

mutable struct GeneralCost{n,m} <: GeneralCostFunction{n,m}
    cost_fn::Function
    is_terminal::Bool
end

function TO.stage_cost(cost::GeneralCost, x::AbstractVector)
    @assert cost.is_terminal
    cost.cost_fn(x)
end

function TO.stage_cost(cost::GeneralCost, x::AbstractVector, u::AbstractVector)
    @assert !cost.is_terminal
    cost.cost_fn(x, u)
end


"""
    gradient!(E::QuadraticCostFunction, costfun::CostFunction, x, u)
    gradient!(E::QuadraticCostFunction, costfun::CostFunction, x)
Evaluate the gradient of the cost function `costfun` at state `x` and control `u`, storing
    the result in `E.q` and `E.r`. Return a `true` if the gradient is constant, and `false`
    otherwise.
"""
function TO.gradient!(E::TO.QuadraticCostFunction, cost::GeneralCostFunction, x::AbstractVector)
    @assert cost.is_terminal
    grad = ForwardDiff.gradient(cost.cost_fn, x)
    E.q .= grad
    return false
end

function TO.gradient!(E::TO.QuadraticCostFunction, cost::GeneralCostFunction, x::AbstractVector, u::AbstractVector)
    @assert !cost.is_terminal
    state_lambda = t -> cost.cost_fn(t, u)
    ctrl_lambda = t -> cost.cost_fn(x, t)
    E.q .= ForwardDiff.gradient(state_lambda, x)
    E.r .= ForwardDiff.gradient(ctrl_lambda, u)
    return false
end

"""
    hessian!(E::QuadraticCostFunction, costfun::CostFunction, x, u)
    hessian!(E::QuadraticCostFunction, costfun::CostFunction, x)
Evaluate the hessian of the cost function `costfun` at state `x` and control `u`, storing
    the result in `E.Q`, `E.R`, and `E.H`. Return a `true` if the hessian is constant, and `false`
    otherwise.
"""
function TO.hessian!(E::TO.QuadraticCostFunction, cost::GeneralCostFunction, x::AbstractVector)
    @assert cost.is_terminal
    E.Q .= ForwardDiff.hessian(cost.cost_fn, x)
    return false
end

function TO.hessian!(E::TO.QuadraticCostFunction, cost::GeneralCostFunction, x::AbstractVector, u::AbstractVector)
    @assert !cost.is_terminal
    state_lambda = t -> cost.cost_fn(t, u)
    ctrl_lambda = t -> cost.cost_fn(x, t)
    E.Q .= ForwardDiff.hessian(state_lambda, x)
    E.R .= ForwardDiff.hessian(ctrl_lambda, u)
    return false
end



In [6]:
import Base
Base.copy(c::GeneralCost) = c

In [7]:
"""
TEST the arbitrary nonlinear cost by comparing to quadratic (LQR) cost
"""

start = SA[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];
finish = SA[1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0];

Q_nom = Diagonal(@SVector fill(1e-5, 7));
R_nom = Diagonal(@SVector fill(1e-3, 7));
Q_f = Diagonal(@SVector fill(100.0, 7));
R_f = R_nom;

n_knotpoint = 50

TO.LQRCost(Q_f, R_f, finish)

lqr_cost_arr = map(1:n_knotpoint) do i
    if i == n_knotpoint
        TO.LQRCost(Q_f, R_f, finish)
    else
        TO.LQRCost(Q_nom, R_nom, finish)
    end
end

my_cost_arr = map(1:n_knotpoint) do i
    if i == n_knotpoint
        GeneralCost{7,7}(x -> TO.stage_cost(lqr_cost_arr[i], x), true)
    else
        GeneralCost{7,7}((x, u) -> TO.stage_cost(lqr_cost_arr[i], x, u), false)
    end
end

obj_lqr = TO.Objective(lqr_cost_arr)
obj_gen = TO.Objective(my_cost_arr)

U0 = [@SVector zeros(7) for _ in 1:n_knotpoint-1]

tf = 10.0 # time (sec)
prob_lqr = Problem(model, obj_lqr, finish, tf, x0=start)
initial_controls!(prob_lqr, U0)
rollout!(prob_lqr);

using Altro
opts = SolverOptions(
    penalty_scaling=100.,
    penalty_initial=0.1,
)

solver = ALTROSolver(prob_lqr, opts);
solve!(solver)
println("Cost: ", cost(solver))

prob_gen = Problem(model, obj_gen, finish, tf, x0=start)
initial_controls!(prob_gen, U0)
rollout!(prob_gen)

solver = ALTROSolver(prob_gen, opts);
solve!(solver)
println("Cost: ", cost(solver))

Cost: 0.0006616597828016417
Cost: 0.0006616597828016417


Next, we need a container cost function that can add together multiple different cost functions at a given knot point

In [8]:
function CompoundCost(cost_list::AbstractVector{TO.CostFunction}, is_terminal::Bool, n, m)
    if is_terminal
        return GeneralCost{n,m}(x -> sum([TO.stage_cost(c, x) for c in cost_list]), is_terminal)
    else
        return GeneralCost{n,m}((x, u) -> sum([TO.stage_cost(c, x, u) for c in cost_list]), is_terminal)
    end
end

CompoundCost (generic function with 1 method)

Add CoMOTO costs:

In [ ]:
function sq_norm(x::AbstractVector)
    dot(x, x)
end

function legibility_cost(x::AbstractVector, u::AbstractVector, jacobian::Function, dt::Float64, curr_timestep::Int, total_timesteps::Int, goal::AbstractVector, start::AbstractVector)
    # exp(-C(s->q) - C*(q->g))/exp(-C*(s->g))
    # = exp(-C(s->q) - C*(q->g) + C(s->g))
    J = jacobian(x);
    start_goal_vel = (goal - start)/total_timesteps;
    nom_goal_vel = (goal - x)/(total_timesteps - curr_timestep);
    curr_vel = J*u;
    exp(-sq_norm(curr_vel) - sq_norm(nom_goal_vel) + sq_norm(start_goal_vel))
end

function visibility_cost(eef_pos::AbstractVector, axis::AbstractVector)
    acos(dot(eef_pos, axis)/(norm(eef_pos)*norm(axis)))
end

function jointwise_distance(cart_joints::AbstactVector, human_pos::AbstactVector)
    
end

function distance(cart_joints::AbstractVector, human_pos::AbstractVector)
    
end

function distance_cost(cart_joints::AbstractVector, human_pos::AbstractVector)
    dist = distance(cart_joints, human_pos);
    
end

function nominal(x::AbstractVector)
    
end

function nominal(x::AbstractVector, u::AbstractVector)